## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
# sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
103
+-------------------+-------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+-------------------+-------------------+------------------+-----------+---------+------+--------------------+------+
|            coeff_1|            coeff_2|            coeff_3|            coeff_4|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|              res_3|              res_4|          res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+-------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+-------------------+-------------------+------------------+-----------+---------+------+--------------------+------+
|-0.9714474436583922|-20.310600885779007|-11.156771769950065|0.05573

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1, coeff_2, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1, coeff_2, coeff_3 FROM weather


,station,year,coeff_1,coeff_2,coeff_3
0,USC00091425,1967.0,-0.971447,-20.310601,-11.156772
1,USC00098661,1967.0,-0.971447,-20.310601,-11.156772
2,USW00003813,1982.0,7.902277,0.964970,0.049238
3,USW00093842,1982.0,7.902277,0.964970,0.049238
4,USC00092198,1967.0,-0.909705,-1.986909,-19.912815


In [7]:
year_station_table1=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table1.head(10)

station,US1GATY0002,USC00010430,USC00011725,USC00012730,USC00014080,USC00014502,USC00016129,USC00017025,USC00091425,USC00092159,...,USC00097201,USC00098535,USC00098657,USC00098661,USC00099124,USC00099291,USC00099506,USW00003813,USW00013860,USW00093842
year,,,,,,,,,,,,,,,,,,,,,
1935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.429300,NaN,NaN,NaN,NaN
1940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.699198,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,13.335233,NaN,NaN,NaN,NaN
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.486062,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.891371,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.057494,NaN,NaN,...,NaN,3.573302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
year_station_table2=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table2.head(10)

station,US1GATY0002,USC00010430,USC00011725,USC00012730,USC00014080,USC00014502,USC00016129,USC00017025,USC00091425,USC00092159,...,USC00097201,USC00098535,USC00098657,USC00098661,USC00099124,USC00099291,USC00099506,USW00003813,USW00013860,USW00093842
year,,,,,,,,,,,,,,,,,,,,,
1935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.974816,NaN,NaN,NaN,NaN
1940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.868259,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.968872,NaN,NaN,NaN,NaN
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.981508,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00544,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.870055,NaN,NaN,...,NaN,0.964797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
year_station_table3=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table3.head(10)

station,US1GATY0002,USC00010430,USC00011725,USC00012730,USC00014080,USC00014502,USC00016129,USC00017025,USC00091425,USC00092159,...,USC00097201,USC00098535,USC00098657,USC00098661,USC00099124,USC00099291,USC00099506,USW00003813,USW00013860,USW00093842
year,,,,,,,,,,,,,,,,,,,,,
1935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.060435,NaN,NaN,NaN,NaN
1940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.132800,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.083156,NaN,NaN,NaN,NaN
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047454,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049258,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.294954,NaN,NaN,...,NaN,-0.064052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [10]:
# ORIGINALITY: Changed RMS to MS. Also, implemented PrintFractionExplained function to explain the fraction by which variance is reduced 
# when we subtract from each station/year entry the average-per-year or the average-per-station respectively. 
# We print these for coeff1, coeff2 and coeff3. These results are also included in the report.

def MS(Mat):
    return np.nanmean(Mat**2)

def PrintFractionExplained(year_station_table):
    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
    tbl_minus_station = year_station_table-mean_by_station
    print 'total MS                   = ',MS(year_station_table)
    print 'MS removing mean-by-station= ',MS(tbl_minus_station)
    print 'Fraction Explained = ',(MS(year_station_table)-MS(tbl_minus_station))/MS(year_station_table)*100
    print 'MS removing mean-by-year   = ',MS(tbl_minus_year)
    print 'Fraction Explained = ',(MS(year_station_table)-MS(tbl_minus_year))/MS(year_station_table)*100
    
PrintFractionExplained(year_station_table1)
PrintFractionExplained(year_station_table2)
PrintFractionExplained(year_station_table3)

total MS                   =  9734.99735929
MS removing mean-by-station=  4443.87675531
Fraction Explained =  54.3515360991
MS removing mean-by-year   =  80.3140511889
Fraction Explained =  99.1749966823
total MS                   =  5689.913501
MS removing mean-by-station=  4012.18198595
Fraction Explained =  29.486063624
MS removing mean-by-year   =  1611.59505356
Fraction Explained =  71.676282016
total MS                   =  731.724942806
MS removing mean-by-station=  477.816011085
Fraction Explained =  34.700051463
MS removing mean-by-year   =  618.131053812
Fraction Explained =  15.5241242096


In [11]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,US1GATY0002,USC00010430,USC00011725,USC00012730,USC00014080,USC00014502,USC00016129,USC00017025,USC00091425,USC00092159,...,USC00097201,USC00098535,USC00098657,USC00098661,USC00099124,USC00099291,USC00099506,USW00003813,USW00013860,USW00093842
year,,,,,,,,,,,,,,,,,,,,,
1935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.974816,NaN,NaN,NaN,NaN
1940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.868259,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.968872,NaN,NaN,NaN,NaN
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.981508,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00544,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.870055,NaN,NaN,...,NaN,0.964797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

T=year_station_table
print 'initial MS=',MS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',MS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',MS(T)

initial MS= 5689.913501
0 after removing mean by year    = 1611.59505356
0 after removing mean by stations= 1319.2008104
1 after removing mean by year    = 1275.48541362
1 after removing mean by stations= 1258.23424324
2 after removing mean by year    = 1249.8074359
2 after removing mean by stations= 1245.20150831
3 after removing mean by year    = 1242.48462623
3 after removing mean by stations= 1240.78799905
4 after removing mean by year    = 1239.67908221
4 after removing mean by stations= 1238.92626394
